In [1]:
import os
import h5py
import numpy as np
import pickle
import autokeras as ak
import tensorflow as tf
import pandas as pd

2021-12-16 15:23:37.152169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-16 15:23:37.152224: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Reading files. 

# change paths here. 

session_train=pd.read_csv('./CSV files/session_train.csv')
session_validation=pd.read_csv('./CSV files/session_val.csv')
session_test=pd.read_csv('./CSV files/final_test.csv')


train_path = './Features/TextualFeatures_csv/time_train.csv'
val_path = './Features/TextualFeatures_csv/time_valid.csv'
test_path = './Features/TextualFeatures_csv/time_test.csv'


data_validation = pd.read_csv(val_path)
data_train = pd.read_csv(train_path)
data_test = pd.read_csv(test_path)

drop_cols = ['Unnamed: 0', 'Video']

data_train.columns
data_validation.drop(drop_cols, axis = 1, inplace=True)
data_train.drop(drop_cols, axis = 1, inplace=True)
data_test.drop(drop_cols, axis = 1, inplace=True)


In [3]:
# data_train.index = 1267
# data_validation.index = 1089

# data_test.index = 489

training_name = train_path.split('/')[3].split('_')[0]+'_'



In [4]:
training_name
# used later to name models and files

'time_'

In [5]:
drop_cols = ['Unnamed: 0', 'Video', 'PART.1', 'PART.2', 'GENDER', 'AGE',]
session_train.drop(drop_cols, axis = 1, inplace=True)
session_validation.drop(drop_cols, axis = 1, inplace=True)
session_test.drop(drop_cols, axis = 1, inplace=True)


In [6]:
# len(session_validation.columns)
session_test.drop(['SESSION1', 'SESSION2', 'SESSION3', 'SESSION4' ], axis = 1, inplace=True)


In [7]:
session_test.head()

,ID_y
0,8
1,56
2,66
3,67
4,86


In [8]:
data_train = pd.merge(session_train, data_train, left_on='ID_y', right_on='ID_y').drop_duplicates()
data_validation = pd.merge(session_validation, data_validation, left_on='ID_y', right_on='ID_y').drop_duplicates()
data_test = pd.merge(session_test, data_test, left_on='ID_y', right_on='ID_y').drop_duplicates()

In [9]:
data_train.head()

,ID_y,OPENMINDEDNESS_Z,CONSCIENTIOUSNESS_Z,EXTRAVERSION_Z,AGREEABLENESS_Z,NEGATIVEEMOTIONALITY_Z,minute,session,gender,age,TimeMin,TimeMax,TimeAvg,TimeStdev,TimeTotal
0,3,-0.263908,0.616152,0.243083,0.982578,-0.437972,0,lego,F,28,760.0,3480.0,1352.941176,657.443934,23000.0
1,3,-0.263908,0.616152,0.243083,0.982578,-0.437972,1,lego,F,28,400.0,2480.0,1280.000000,485.084185,19200.0
2,3,-0.263908,0.616152,0.243083,0.982578,-0.437972,2,lego,F,28,760.0,3520.0,1708.571429,860.355883,23920.0
3,3,-0.263908,0.616152,0.243083,0.982578,-0.437972,3,lego,F,28,880.0,3400.0,1694.736842,783.536977,32200.0
4,3,-0.263908,0.616152,0.243083,0.982578,-0.437972,4,lego,F,28,600.0,2600.0,1328.000000,483.189404,33200.0


In [10]:
labels = ['OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z', 'EXTRAVERSION_Z', 'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z']


In [11]:
M_train = data_train[data_train['gender'] == 'M']
F_train = data_train[data_train['gender'] == 'F']

M_val = data_validation[data_validation['gender'] == 'M']
F_val = data_validation[data_validation['gender'] == 'F']

M_test = data_test[data_test['gender'] == 'M']
F_test = data_test[data_test['gender'] == 'F']

In [12]:
Y_train = data_train[data_train['age'] <= 30]
O_train = data_train[data_train['age'] > 30]

Y_val = data_validation[data_validation['age'] <= 30]
O_val = data_validation[data_validation['age'] > 30]

Y_test = data_test[data_test['age'] <= 30]
O_test = data_test[data_test['age'] > 30]

In [13]:
Y_train.columns

Index(['ID_y', 'OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z', 'EXTRAVERSION_Z',
       'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z', 'minute', 'session',
       'gender', 'age', 'TimeMin', 'TimeMax', 'TimeAvg', 'TimeStdev',
       'TimeTotal'],
      dtype='object')

In [14]:
print(data_train[data_train['age'] <= 30].shape)
print(data_validation[data_validation['age'] <= 30].shape)
print(Y_val.shape)

(680, 15)
(870, 15)
(870, 15)


In [15]:
M_train = M_train.append(M_val)
F_train = F_train.append(F_val)

In [16]:
Y_train = Y_train.append(Y_val)
O_train = O_train.append(O_val)

In [17]:
Y_train.shape

(1550, 15)

In [18]:
df_train = data_train.append(data_validation)
df_validation = data_validation
df_test = data_test

In [19]:
# Combined 
train_O = df_train.pop(labels[0]).to_numpy()
train_C = df_train.pop(labels[1]).to_numpy()
train_E = df_train.pop(labels[2]).to_numpy()
train_A = df_train.pop(labels[3]).to_numpy()
train_N = df_train.pop(labels[4]).to_numpy()

val_O = df_validation.pop(labels[0]).to_numpy()
val_C = df_validation.pop(labels[1]).to_numpy()
val_E = df_validation.pop(labels[2]).to_numpy()
val_A = df_validation.pop(labels[3]).to_numpy()
val_N = df_validation.pop(labels[4]).to_numpy()

# male train OCEAN 
M_train_O = M_train.pop(labels[0]).to_numpy()
M_train_C = M_train.pop(labels[1]).to_numpy()
M_train_E = M_train.pop(labels[2]).to_numpy()
M_train_A = M_train.pop(labels[3]).to_numpy()
M_train_N= M_train.pop(labels[4]).to_numpy()

# female train OCEAN
F_train_O = F_train.pop(labels[0]).to_numpy()
F_train_C = F_train.pop(labels[1]).to_numpy()
F_train_E = F_train.pop(labels[2]).to_numpy()
F_train_A = F_train.pop(labels[3]).to_numpy()
F_train_N= F_train.pop(labels[4]).to_numpy()

# male validation OCEAN

M_val_O = M_val.pop(labels[0]).to_numpy()
M_val_C = M_val.pop(labels[1]).to_numpy()
M_val_E = M_val.pop(labels[2]).to_numpy()
M_val_A = M_val.pop(labels[3]).to_numpy()
M_val_N = M_val.pop(labels[4]).to_numpy()

# female validation OCEAN

F_val_O = F_val.pop(labels[0]).to_numpy()
F_val_C = F_val.pop(labels[1]).to_numpy()
F_val_E = F_val.pop(labels[2]).to_numpy()
F_val_A = F_val.pop(labels[3]).to_numpy()
F_val_N = F_val.pop(labels[4]).to_numpy()

In [20]:


# male train OCEAN 
Y_train_O = Y_train.pop(labels[0]).to_numpy()
Y_train_C = Y_train.pop(labels[1]).to_numpy()
Y_train_E = Y_train.pop(labels[2]).to_numpy()
Y_train_A = Y_train.pop(labels[3]).to_numpy()
Y_train_N=  Y_train.pop(labels[4]).to_numpy()

# female train OCEAN
O_train_O = O_train.pop(labels[0]).to_numpy()
O_train_C = O_train.pop(labels[1]).to_numpy()
O_train_E = O_train.pop(labels[2]).to_numpy()
O_train_A = O_train.pop(labels[3]).to_numpy()
O_train_N=  O_train.pop(labels[4]).to_numpy()

# male validation OCEAN

Y_val_O = Y_val.pop(labels[0]).to_numpy()
Y_val_C = Y_val.pop(labels[1]).to_numpy()
Y_val_E = Y_val.pop(labels[2]).to_numpy()
Y_val_A = Y_val.pop(labels[3]).to_numpy()
Y_val_N = Y_val.pop(labels[4]).to_numpy()

# female validation OCEAN

O_val_O = O_val.pop(labels[0]).to_numpy()
O_val_C = O_val.pop(labels[1]).to_numpy()
O_val_E = O_val.pop(labels[2]).to_numpy()
O_val_A = O_val.pop(labels[3]).to_numpy()
O_val_N = O_val.pop(labels[4]).to_numpy()

In [21]:
data_cols = [ 'ID_y', 'minute']
drop_cols = []

train_data = df_train[data_cols]
val_data   = df_validation[data_cols]
test_data  = df_test[data_cols]

Y_train_data = Y_train[data_cols]
Y_val_data   = Y_val[data_cols]
Y_test_data  = Y_test[data_cols]

O_train_data = O_train[data_cols]
O_val_data   = O_val[data_cols]
O_test_data  = O_test[data_cols]

M_train_data = M_train[data_cols]
F_train_data = F_train[data_cols]
M_val_data = M_val[data_cols]
F_val_data = F_val[data_cols]
M_test_data = M_test[data_cols]
F_test_data = F_test[data_cols]

In [22]:
Y_train_data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,   48,   49,
            ...
            2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160],
           dtype='int64', length=1550)

In [23]:
data_cols = ['ID_y','minute', 'session', 'gender', 'age']
# combined
train_com = df_train.drop(data_cols+drop_cols, axis = 1)
val_com   = df_validation.drop(data_cols+drop_cols, axis = 1)
test_com  = df_test.drop(data_cols+drop_cols, axis = 1)



# Age
Y_train = Y_train.drop(data_cols+drop_cols, axis = 1)
Y_val   = Y_val.drop(data_cols+drop_cols, axis = 1)
Y_test  = Y_test.drop(data_cols+drop_cols, axis = 1)

O_train = O_train.drop(data_cols+drop_cols, axis = 1)
O_val   = O_val.drop(data_cols+drop_cols, axis = 1)
O_test  = O_test.drop(data_cols+drop_cols, axis = 1)



# Gender
M_train = M_train.drop(data_cols+drop_cols, axis = 1)
M_val = M_val.drop(data_cols+drop_cols, axis = 1)
M_test = M_test.drop(data_cols+drop_cols, axis = 1)

F_train = F_train.drop(data_cols+drop_cols, axis = 1)
F_val = F_val.drop(data_cols+drop_cols, axis = 1)
F_test = F_test.drop(data_cols+drop_cols, axis = 1)



In [24]:
# Comvined
train_data.reset_index(drop = True, inplace = True)
val_data.reset_index(drop = True, inplace = True)
test_data.reset_index(drop = True, inplace = True)


# Age

Y_train.reset_index(drop=True, inplace=True)
Y_train_data.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
Y_test_data.reset_index(drop = True, inplace = True)
Y_val.reset_index(drop=True, inplace=True)
Y_val_data.reset_index(drop=True, inplace=True)

O_train.reset_index(drop=True, inplace=True)
O_train_data.reset_index(drop=True, inplace=True)
O_val.reset_index(drop=True, inplace=True)
O_val_data.reset_index(drop=True, inplace=True)
O_test.reset_index(drop=True, inplace=True)
O_test_data.reset_index(drop = True, inplace = True)

# Gender
M_train.reset_index(drop=True, inplace=True)
M_train_data.reset_index(drop=True, inplace=True)
M_test.reset_index(drop=True, inplace=True)
M_test_data.reset_index(drop = True, inplace = True)
M_val.reset_index(drop=True, inplace=True)
M_val_data.reset_index(drop=True, inplace=True)

F_train.reset_index(drop=True, inplace=True)
F_train_data.reset_index(drop=True, inplace=True)
F_val.reset_index(drop=True, inplace=True)
F_val_data.reset_index(drop=True, inplace=True)
F_test.reset_index(drop=True, inplace=True)
F_test_data.reset_index(drop = True, inplace = True)

In [25]:
train_data.shape

(2359, 2)

In [26]:
train_C.shape

(2359,)

In [27]:

OCEAN_models = ['Model_O', 'Model_C', 'Model_E', 'Model_A', 'Model_N']
# Combined
train_com_np = np.array(train_com)
val_com_np = np.array(val_com)
test_com_np = np.array(test_com)

# Age
Y_train_np = np.array(Y_train)
Y_test_np  = np.array(Y_test)
Y_val_np   = np.array(Y_val)


O_train_np = np.array(O_train)
O_val_np   = np.array(O_val)
O_test_np  = np.array(O_test)

# GEnder
M_train_np = np.array(M_train)
M_test_np  = np.array(M_test)
M_val_np   = np.array(M_val)


F_train_np = np.array(F_train)
F_val_np   = np.array(F_val)
F_test_np  = np.array(F_test)



In [28]:
# Combined
comb_O_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_O))
comb_C_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_C))
comb_E_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_E))
comb_A_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_A))
comb_N_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_N))

comb_O_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_O))
comb_C_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_C))
comb_E_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_E))
comb_A_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_A))
comb_N_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_N))


# Age

youngO_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_O))
youngC_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_C))
youngE_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_E))
youngA_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_A))
youngN_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_N))
youngO_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_O))
youngC_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_C))
youngE_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_E))
youngA_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_A))
youngN_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_N))

oldO_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_O))
oldC_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_C))
oldE_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_E))
oldA_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_A))
oldN_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_N))
oldO_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_O))
oldC_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_C))
oldE_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_E))
oldA_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_A))
oldN_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_N))


# Gender
maleO_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_O))
maleC_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_C))
maleE_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_E))
maleA_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_A))
maleN_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_N))

femaleO_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_O))
femaleC_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_C))
femaleE_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_E))
femaleA_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_A))
femaleN_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_N))

maleO_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_O))
maleC_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_C))
maleE_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_E))
maleA_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_A))
maleN_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_N))

femaleO_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_O))
femaleC_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_C))
femaleE_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_E))
femaleA_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_A))
femaleN_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_N))


2021-12-16 15:23:42.998378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-16 15:23:42.998442: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-16 15:23:42.998483: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vish-Lenovo-Yoga-S730-13IWL): /proc/driver/nvidia/version does not exist
2021-12-16 15:23:42.998983: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
# combined
comb_train_set = [comb_O_train_set, 
                  comb_C_train_set, 
                  comb_E_train_set, 
                  comb_A_train_set, 
                  comb_N_train_set]

comb_validation_set = [comb_O_validation_set, 
                       comb_C_validation_set, 
                       comb_E_validation_set, 
                       comb_A_validation_set, 
                       comb_N_validation_set]

# Age
young_train_set =     [youngO_train_set, 
                       youngC_train_set, 
                       youngE_train_set, 
                       youngA_train_set, 
                       youngN_train_set]
young_validation_set =[youngO_validation_set, 
                       youngC_validation_set, 
                       youngE_validation_set, 
                       youngA_validation_set, 
                       youngN_validation_set]

old_train_set =         [oldO_train_set, 
                         oldC_train_set, 
                         oldE_train_set,
                         oldA_train_set,
                         oldN_train_set]
old_validation_set =    [oldO_validation_set,
                         oldC_validation_set,
                         oldE_validation_set,
                         oldA_validation_set,
                         oldN_validation_set]

# Gender
male_train_set =      [maleO_train_set, 
                       maleC_train_set, 
                       maleE_train_set, 
                       maleA_train_set, 
                       maleN_train_set]
male_validation_set = [maleO_validation_set, 
                       maleC_validation_set, 
                       maleE_validation_set, 
                       maleA_validation_set, 
                       maleN_validation_set]

female_train_set =      [femaleO_train_set, 
                         femaleC_train_set, 
                         femaleE_train_set,
                         femaleA_train_set,
                         femaleN_train_set]
female_validation_set = [femaleO_validation_set,
                         femaleC_validation_set,
                         femaleE_validation_set,
                         femaleA_validation_set,
                         femaleN_validation_set]

model_names = ['Model_O','Model_C','Model_E','Model_A','Model_N']


In [30]:
training_name

'time_'

# Make folders

In [31]:
dump = './Dump Data'
path = './Textual Models'
models_path = path+'/'+training_name
combined_path = models_path+'/Combined'
age_path = models_path+'/Age'
gender_path = models_path+'/Gender'
male_path = models_path+'/Gender/Male'
female_path = models_path+'/Gender/Female'
old_path = models_path+'/Age/Old'
young_path = models_path+'/Age/Young'

folders = [ dump
,path
,models_path
,combined_path
,age_path
,gender_path
,male_path
,female_path
,old_path
,young_path]

In [32]:
for folder in folders:
    try:
        os.mkdir(folder)
    except OSError as error:
        print(error)  

[Errno 17] File exists: './Dump Data'
[Errno 17] File exists: './Textual Models'
[Errno 17] File exists: './Textual Models/time_'
[Errno 17] File exists: './Textual Models/time_/Combined'
[Errno 17] File exists: './Textual Models/time_/Age'
[Errno 17] File exists: './Textual Models/time_/Gender'
[Errno 17] File exists: './Textual Models/time_/Gender/Male'
[Errno 17] File exists: './Textual Models/time_/Gender/Female'
[Errno 17] File exists: './Textual Models/time_/Age/Old'
[Errno 17] File exists: './Textual Models/time_/Age/Young'


# Training

## Combined

In [33]:
"""# Train
Currently:  
Validation split: 0.15  
Epochs: 1000  
Trials: 100  
## Combined models
"""
print("Combined TRAINING BEGINS")
for i in range(5):
  train = comb_train_set[i]
  val = comb_validation_set[i]
  print(model_names[i])
  # Define a regressor
  total_reg = ak.StructuredDataRegressor(max_trials=100, overwrite=True,project_name = training_name+model_names[i], directory = './Dump Data')
  # Feed the tensorflow Dataset to the regressor.
  total_reg.fit(train, epochs=1000, validation_split=0.15)
  # Convert to model   
  total_model = total_reg.export_model()
  # Evaluate on validation set
  evaluation = total_reg.evaluate(val)
  # Write loss and error to a file
  with open('./Dump Data/'+training_name+'combined_eval_val.txt', 'a') as f:
      f.write(training_name+'combined_'+model_names[i]+' -> ')
      f.write(str(evaluation))
      f.write('\n')
  # Save current model
  total_model.save(combined_path+'/'+model_names[i])

print("Combined training done!!")

Trial 89 Complete [00h 00m 12s]
val_loss: 0.3853585720062256

Best val_loss So Far: 0.3738803565502167
Total elapsed time: 00h 17m 48s
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
74/74 [==============================] - 1s 5ms/step - loss: 2.4651 - mean_squared_error: 2.4651
Epoch 2/1000
74/74 [==============================] - 0s 4ms/step - loss: 2.2878 - mean_squared_error: 2.2878
Epoch 3/1000
74/74 [==============================] - 0s 3ms/step - loss: 2.3656 - mean_squared_error: 2.3656
Epoch 4/1000
74/74 [==============================] - 0s 4ms/step - loss: 2.3024 - mean_squared_error: 2.3024
Epoch 5/1000
74/74 [==============================] - 0s 5ms/step - loss: 2.2900 - mean_squared_error: 2.2900
Epoch 6/1000
74/74 [==============================] - 0s 5ms/step - loss: 2.2941 - mean_squared_error: 2.2941
Epoch 7/1000
74/74 [==============================] - 0s 5ms/step - loss: 2.3109 - mean_squared_error: 2.3109
Epoch 8/1000
74/74 [==============================] - 0s 

## Age

In [34]:
"""# Train
Currently:  
Validation split: 0.15  
Epochs: 1000  
Trials: 100  
## Age models
"""
print("Young TRAINING BEGINS")
for i in range(5):
  train = young_train_set[i]
  val = young_validation_set[i]
  print(model_names[i])
  # Define a regressor
  total_reg = ak.StructuredDataRegressor(max_trials=100, overwrite=True,project_name = training_name+'young_'+model_names[i], directory = './Dump Data')
  # Feed the tensorflow Dataset to the regressor.
  total_reg.fit(train, epochs=1000, validation_split=0.15)
  # Convert to model   
  total_model = total_reg.export_model()
  # Evaluate on validation set
  evaluation = total_reg.evaluate(val)
  # Write loss and error to a file
  with open('./Dump Data/'+training_name+'young_eval_val.txt', 'a') as f:
      f.write(training_name+'young_'+model_names[i]+' -> ')
      f.write(str(evaluation))
      f.write('\n')
  # Save current model
  total_model.save(young_path+'/'+model_names[i])

# """## Old models"""
print("Old TRAINING BEGINS")

for i in range(5):
  train = old_train_set[i]
  val = old_validation_set[i]
  # Define a regressor
  total_reg = ak.StructuredDataRegressor(max_trials=100, overwrite=True,project_name = training_name+'old_'+model_names[i], directory = './Dump Data')
  # Feed the tensorflow Dataset to the regressor.
  total_reg.fit(train, epochs=1000, validation_split=0.15)
  # Convert to model   
  total_model = total_reg.export_model()
  # Evaluate on validation set
  evaluation = total_reg.evaluate(val)
  # Write loss and error to a file
  with open('./Dump Data/'+training_name+'old_eval_val.txt', 'a') as f:
      f.write(training_name+'old_'+model_names[i]+' -> ')
      f.write(str(evaluation))
      f.write('\n')
  # Save current model  
  total_model.save(old_path+'/'+model_names[i])
print("Age TRAINING DONE")

Trial 42 Complete [00h 00m 24s]
val_loss: 152.7230682373047

Best val_loss So Far: 0.3774683177471161
Total elapsed time: 00h 06m 21s
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
26/26 [==============================] - 1s 3ms/step - loss: 1.4432 - mean_squared_error: 1.4432
Epoch 2/1000
26/26 [==============================] - 0s 3ms/step - loss: 1.1884 - mean_squared_error: 1.1884
Epoch 3/1000
26/26 [==============================] - 0s 3ms/step - loss: 1.1114 - mean_squared_error: 1.1114
Epoch 4/1000
26/26 [==============================] - 0s 3ms/step - loss: 1.0620 - mean_squared_error: 1.0620
Epoch 5/1000
26/26 [==============================] - 0s 3ms/step - loss: 1.0509 - mean_squared_error: 1.0509
Epoch 6/1000
26/26 [==============================] - 0s 4ms/step - loss: 1.0301 - mean_squared_error: 1.0301
Epoch 7/1000
26/26 [==============================] - 0s 3ms/step - loss: 1.0475 - mean_squared_error: 1.0475
Epoch 8/1000
26/26 [==============================] - 0s 3

## Gender

In [35]:
"""# Train
Currently:  
Validation split: 0.15  
Epochs: 1000  
Trials: 100  
## Age models
"""
print("Male TRAINING BEGINS")
for i in range(5):
  train = male_train_set[i]
  val = male_validation_set[i]
  print(model_names[i])
  # Define a regressor
  total_reg = ak.StructuredDataRegressor(max_trials=100, overwrite=True,project_name = training_name+'male_'+model_names[i], directory = './Dump Data')
  # Feed the tensorflow Dataset to the regressor.
  total_reg.fit(train, epochs=1000, validation_split=0.15)
  # Convert to model   
  total_model = total_reg.export_model()
  # Evaluate on validation set
  evaluation = total_reg.evaluate(val)
  # Write loss and error to a file
  with open('./Dump Data/'+training_name+'male_eval_val.txt', 'a') as f:
      f.write(training_name+'male_'+model_names[i]+' -> ')
      f.write(str(evaluation))
      f.write('\n')
  # Save current model
  total_model.save(male_path+'/'+model_names[i])

# """## Female models"""
print("Female TRAINING BEGINS")

for i in range(5):
  train = female_train_set[i]
  val = female_validation_set[i]
  # Define a regressor
  total_reg = ak.StructuredDataRegressor(max_trials=100, overwrite=True,project_name = training_name+'female_'+model_names[i], directory = './Dump Data')
  # Feed the tensorflow Dataset to the regressor.
  total_reg.fit(train, epochs=1000, validation_split=0.15)
  # Convert to model   
  total_model = total_reg.export_model()
  # Evaluate on validation set
  evaluation = total_reg.evaluate(val)
  # Write loss and error to a file
  with open('./Dump Data/'+training_name+'female_eval_val.txt', 'a') as f:
      f.write(training_name+'female_'+model_names[i]+' -> ')
      f.write(str(evaluation))
      f.write('\n')
  # Save current model  
  total_model.save(female_path+'/'+model_names[i])
print("Gender TRAINING DONE")

Trial 55 Complete [00h 00m 04s]
val_loss: 1.9697010517120361

Best val_loss So Far: 1.7518939971923828
Total elapsed time: 00h 57m 51s
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
36/36 [==============================] - 1s 3ms/step - loss: 1.5060 - mean_squared_error: 1.5060
Epoch 2/1000
36/36 [==============================] - 0s 2ms/step - loss: 1.5044 - mean_squared_error: 1.5044
Epoch 3/1000
36/36 [==============================] - 0s 3ms/step - loss: 1.5031 - mean_squared_error: 1.5031
Epoch 4/1000
36/36 [==============================] - 0s 2ms/step - loss: 1.5017 - mean_squared_error: 1.5017
Epoch 5/1000
36/36 [==============================] - 0s 2ms/step - loss: 1.5004 - mean_squared_error: 1.5004
Epoch 6/1000
36/36 [==============================] - 0s 2ms/step - loss: 1.4990 - mean_squared_error: 1.4990
Epoch 7/1000
36/36 [==============================] - 0s 2ms/step - loss: 1.4977 - mean_squared_error: 1.4977
Epoch 8/1000
36/36 [==============================] - 0s 